In [1]:
!pip install -q tensorflow

import numpy as np
import tensorflow as tf
import random as rn

In [2]:
import os
os.environ['PYTHONHASHSEED'] = '0'

np.random.seed(37)
rn.seed(1254)
tf.random.set_seed(89)

In [3]:
from tensorflow.python.keras import backend as K

# force tensorflow to use a single thread
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess=tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(),config=config)
K.set_session(sess)

In [4]:
## Imports libs
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ["SM_FRAMEWORK"] = "tf.keras"

from tensorflow import keras
import segmentation_models as sm

from tensorflow.keras import layers
from keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam

Segmentation Models: using `tf.keras` framework.


Using TensorFlow backend.


In [5]:
!pip install -U -q segmentation-models
!pip install -q keras
!pip install -q tensorflow-estimator

import pandas as pd
import cv2
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from PIL import Image
from random import shuffle
from tqdm import tqdm
import pandas as pd
from sklearn.utils import class_weight

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix

from tensorflow.keras.models import Model,Sequential, load_model,model_from_json
from tensorflow.keras.applications import mobilenet,mobilenet_v2,densenet,inception_resnet_v2,inception_v3,vgg16,resnet_v2,resnet
#from tensorflow.keras.utils.generic_utils import CustomObjectScope
from tensorflow.keras.layers import Flatten, Dropout,GlobalAveragePooling2D,Activation, Conv2D, Reshape,DepthwiseConv2D,Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, Callback, CSVLogger, EarlyStopping
from tensorflow.keras.metrics import top_k_categorical_accuracy
from sklearn.metrics import confusion_matrix

In [6]:
BATCH_SIZE=80
INPUT_SIZE = (224, 224)

In [7]:
model=load_model(r'C:\Users\HP\FER_SL\models\affectnet_emotions\mobilenet_7.h5')
model.summary()
net_description='mobilenet_7'
def mobilenet_preprocess_input(x,**kwargs):
    x[..., 0] -= 103.939
    x[..., 1] -= 116.779
    x[..., 2] -= 123.68
    return x
preprocessing_function=mobilenet_preprocess_input
        
BATCH_SIZE=80
INPUT_SIZE = (224, 224)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [8]:
net_description='FT_IP_'+net_description

In [9]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocessing_function,horizontal_flip = False)

test_datagen = ImageDataGenerator(preprocessing_function=preprocessing_function)

In [10]:
target_size = (224, 224)

training_set = train_datagen.flow_from_directory(r'C:\Users\HP\FER_SL\sl_data\train_files_cropped', target_size=target_size,
                                                 batch_size=BATCH_SIZE, class_mode='categorical')

test_set = test_datagen.flow_from_directory(r'C:\Users\HP\FER_SL\sl_data\test_files_cropped', target_size=target_size,
                                            batch_size=BATCH_SIZE, class_mode='categorical')

Found 2024 images belonging to 7 classes.
Found 507 images belonging to 7 classes.


# Downloading the previous model

In [11]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [12]:
def count_params(model):
    non_trainable_params = np.sum([np.prod(v.get_shape().as_list()) for v in model.non_trainable_weights])
    trainable_params = np.sum([np.prod(v.get_shape().as_list()) for v in model.trainable_weights])
    return {'non_trainable_params': non_trainable_params, 'trainable_params': trainable_params}

In [13]:
params = count_params(model)
assert params['non_trainable_params'] == 21888
assert params['trainable_params'] == 3471175

# Removing the last layer

In [14]:
new_model = Sequential()
for layer in model.layers[:-1]:
    new_model.add(layer)

# Freezing the previous layers

In [15]:
for layer in new_model.layers:
    layer.trainable = False

# Adding a new dense layer

In [16]:
print(model.input)

KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")


In [17]:
new_model.add(Dense(units=7, input_shape=(224,224), activation='softmax'))
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0

# Training only the last layer

In [18]:
params = count_params(new_model)
assert params['non_trainable_params'] == 3491264
# only the parameters in the last layer are trainable
assert params['trainable_params'] == 1799

In [19]:
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(net_description)
nb_train_samples = 2024
nb_validation_samples = 507

mc = ModelCheckpoint(net_description+'.h5', monitor='val_accuracy', verbose=1, save_best_only=True)
es=EarlyStopping(monitor='val_accuracy',patience=2)
FIRST_EPOCHS=3

hist1=new_model.fit(training_set, steps_per_epoch=nb_train_samples//BATCH_SIZE, epochs=FIRST_EPOCHS, verbose=1, 
                    initial_epoch=0, callbacks=[mc, es], validation_data=test_set, validation_steps=nb_validation_samples // BATCH_SIZE)

FT_IP_mobilenet_7
Epoch 1/3
25/25 [==============================] - 56s 2s/step - loss: 2.3211 - accuracy: 0.1106 - val_loss: 1.9260 - val_accuracy: 0.1604

Epoch 00001: val_accuracy improved from -inf to 0.16042, saving model to FT_IP_mobilenet_7.h5
Epoch 2/3
25/25 [==============================] - 53s 2s/step - loss: 1.7574 - accuracy: 0.3164 - val_loss: 1.6016 - val_accuracy: 0.3833

Epoch 00002: val_accuracy improved from 0.16042 to 0.38333, saving model to FT_IP_mobilenet_7.h5
Epoch 3/3
25/25 [==============================] - 53s 2s/step - loss: 1.5663 - accuracy: 0.4275 - val_loss: 1.4401 - val_accuracy: 0.4792

Epoch 00003: val_accuracy improved from 0.38333 to 0.47917, saving model to FT_IP_mobilenet_7.h5


# Re-training the model with all the layers

In [20]:
start_epoch=len(hist1.history['loss'])

for layer in new_model.layers:
    layer.trainable = True
    
new_model.summary()

params = count_params(new_model)
assert params['non_trainable_params'] == 21888
# only the parameters in the last layer are trainable
assert params['trainable_params'] == 3471175 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0

In [21]:
new_model.load_weights(net_description+'.h5')
new_model.compile(optimizer=Adam(learning_rate=1e-4,decay=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
SECOND_EPOCHS=start_epoch+5

mc = ModelCheckpoint(net_description+'_ft.h5', monitor='val_accuracy', verbose=1, save_best_only=True)
#es=EarlyStopping(monitor='val_accuracy',patience=2 )

print('All tuning')
hist2=new_model.fit(training_set, steps_per_epoch=training_set.samples//BATCH_SIZE, epochs=SECOND_EPOCHS, verbose=1, 
                    initial_epoch=start_epoch, validation_data=test_set, validation_steps=test_set.samples // BATCH_SIZE, callbacks=[mc])

All tuning
Epoch 4/8
25/25 [==============================] - 256s 10s/step - loss: 1.4044 - accuracy: 0.5067 - val_loss: 1.3166 - val_accuracy: 0.4583

Epoch 00004: val_accuracy improved from -inf to 0.45833, saving model to FT_IP_mobilenet_7_ft.h5
Epoch 5/8
25/25 [==============================] - 244s 10s/step - loss: 1.0602 - accuracy: 0.6219 - val_loss: 1.1366 - val_accuracy: 0.5562

Epoch 00005: val_accuracy improved from 0.45833 to 0.55625, saving model to FT_IP_mobilenet_7_ft.h5
Epoch 6/8
25/25 [==============================] - 235s 9s/step - loss: 0.8281 - accuracy: 0.7104 - val_loss: 1.0235 - val_accuracy: 0.6271

Epoch 00006: val_accuracy improved from 0.55625 to 0.62708, saving model to FT_IP_mobilenet_7_ft.h5
Epoch 7/8
25/25 [==============================] - 250s 10s/step - loss: 0.6604 - accuracy: 0.7720 - val_loss: 0.9456 - val_accuracy: 0.6646

Epoch 00007: val_accuracy improved from 0.62708 to 0.66458, saving model to FT_IP_mobilenet_7_ft.h5
Epoch 8/8
25/25 [========

In [23]:
new_model.load_weights(net_description+'_ft.h5')

In [24]:
new_model.compile(optimizer=SGD(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
#new_model.summary()
mc = ModelCheckpoint(net_description+'_ft_sgd.h5', monitor='val_accuracy', verbose=1, save_best_only=True)
second_epoch=start_epoch+len(hist2.history['loss'])
THIRD_EPOCHS=2+second_epoch

print('ALL SGD tuning')
hist3=new_model.fit_generator(training_set, steps_per_epoch=training_set.samples//BATCH_SIZE, epochs=THIRD_EPOCHS, verbose=1, 
                    initial_epoch=second_epoch, validation_data=test_set, validation_steps=test_set.samples // BATCH_SIZE,callbacks=[mc])

ALL SGD tuning


c:\program files\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 9/10
25/25 [==============================] - 247s 10s/step - loss: 0.5360 - accuracy: 0.8313 - val_loss: 0.9395 - val_accuracy: 0.6646

Epoch 00009: val_accuracy improved from -inf to 0.66458, saving model to FT_IP_mobilenet_7_ft_sgd.h5
Epoch 10/10
25/25 [==============================] - 237s 9s/step - loss: 0.5536 - accuracy: 0.8230 - val_loss: 0.9192 - val_accuracy: 0.6583

Epoch 00010: val_accuracy did not improve from 0.66458


In [25]:
USE_PREPROCESSING=True
print(INPUT_SIZE)

(224, 224)


In [26]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [27]:
import numpy as np
import cv2
y_val=np.array(test_set.labels)
y_scores_val=[]
#new_model.load_weights(net_description+'_ft.h5')
for filepath in log_progress(test_set.filepaths):
    img=cv2.imread(filepath)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,INPUT_SIZE)
    if USE_PREPROCESSING:
        inp = preprocessing_function(np.expand_dims(img, axis=0).astype(np.float32))
    else:
        inp=img.astype(np.float32)
        inp[..., 0] -= 103.939
        inp[..., 1] -= 116.779
        inp[..., 2] -= 123.68
        inp = np.expand_dims(inp, axis=0)

    scores=new_model.predict(inp)
    print(scores)
    y_scores_val.append(scores)

y_scores_val=np.concatenate(y_scores_val, axis=0)
print(y_scores_val.shape,y_val.shape)

[[0.1886071  0.00352542 0.13425475 0.00594728 0.05894931 0.02167695
  0.5870392 ]]
[[0.37461528 0.02731694 0.1692506  0.00969701 0.08098119 0.12395174
  0.21418726]]
[[0.37142962 0.02097189 0.1953384  0.0129557  0.09709173 0.17647548
  0.12573713]]
[[9.4899195e-01 2.4292883e-02 1.8341960e-02 4.8918329e-04 1.6557343e-03
  4.9600570e-04 5.7321801e-03]]
[[0.7921826  0.02752992 0.06000151 0.00197145 0.00977514 0.00228292
  0.10625645]]
[[0.9079995  0.01565607 0.03542959 0.00193809 0.00658351 0.00151158
  0.03088169]]
[[0.7353857  0.05707894 0.08367147 0.00650226 0.02169155 0.00293578
  0.09273432]]
[[6.5711266e-01 6.6550742e-03 1.1514479e-01 1.6238253e-03 6.1539193e-03
  2.4396052e-04 2.1306583e-01]]
[[8.4564686e-01 1.4166491e-02 5.9619077e-02 1.5055162e-03 6.5421411e-03
  4.2187906e-04 7.2098143e-02]]
[[8.1876868e-01 7.5207199e-03 8.1054099e-02 2.6691111e-03 6.8449867e-03
  6.5496023e-04 8.2487360e-02]]
[[0.6628898  0.03087748 0.17151259 0.00634318 0.00976539 0.00105552
  0.11755604]]
[[0

[[0.6111297  0.15888539 0.06078636 0.02377879 0.05037225 0.01745113
  0.07759637]]
[[9.4937229e-01 5.4554921e-03 9.3144621e-04 1.3344209e-03 3.0080040e-03
  3.8101140e-02 1.7972167e-03]]
[[9.6439934e-01 7.8473026e-03 4.6759486e-04 8.9518429e-04 1.9856594e-03
  2.2787951e-02 1.6169697e-03]]
[[9.4176245e-01 2.2631217e-02 6.7851879e-03 5.8741827e-04 1.0357391e-03
  2.1101188e-02 6.0967156e-03]]
[[0.86310506 0.04681291 0.00495418 0.00116342 0.00324964 0.07274898
  0.00796585]]
[[0.8523025  0.048923   0.01041068 0.00223008 0.00545367 0.0655873
  0.01509286]]
[[0.8980605  0.02984508 0.00643451 0.0011279  0.0032966  0.04938203
  0.01185337]]
[[0.851111   0.04280071 0.0095945  0.00146532 0.0038835  0.07632454
  0.01482042]]
[[0.9047044  0.01969576 0.00831235 0.00100038 0.00301175 0.04747922
  0.01579613]]
[[8.8115323e-01 2.1606803e-02 1.0497182e-02 7.5715588e-04 2.3483830e-03
  7.3318556e-02 1.0318552e-02]]
[[0.7780599  0.05337317 0.01722587 0.00197119 0.00447031 0.13122964
  0.01366981]]
[[0.

[[0.2504623  0.237453   0.34480354 0.05015458 0.04248009 0.02168726
  0.05295928]]
[[0.19691096 0.16577747 0.46593577 0.03381227 0.02788378 0.03484033
  0.07483939]]
[[0.44415727 0.467782   0.03638053 0.01627624 0.0233614  0.00688305
  0.0051595 ]]
[[0.33474952 0.5333177  0.06068366 0.00200945 0.00844008 0.04357832
  0.01722129]]
[[1.2697866e-03 4.1324938e-06 3.3776872e-02 7.1074173e-05 5.6251569e-04
  5.8792812e-06 9.6430969e-01]]
[[0.01049818 0.00439604 0.12512799 0.0059022  0.10338731 0.07522304
  0.6754653 ]]
[[0.01061616 0.00359665 0.13390714 0.00615775 0.12883356 0.07931616
  0.63757265]]
[[0.02048901 0.00190174 0.15425059 0.01787101 0.03076431 0.00370485
  0.77101856]]
[[0.00870253 0.03952734 0.19852158 0.16329898 0.07508503 0.02031114
  0.49455342]]
[[0.07459201 0.03191272 0.25847283 0.04366926 0.12291906 0.19144744
  0.2769867 ]]
[[0.18885289 0.02674314 0.48575938 0.00273479 0.01676145 0.03676365
  0.24238475]]
[[0.07876049 0.01641942 0.5253633  0.00758094 0.01450366 0.0061470

[[0.7328221  0.01927398 0.03151734 0.0031664  0.05058433 0.06436247
  0.09827336]]
[[2.2171431e-03 1.0003850e-03 1.9713700e-01 5.0853140e-04 9.7688362e-03
  7.2732973e-01 6.2038302e-02]]
[[1.0378835e-03 2.9671186e-04 2.5200343e-02 1.9218567e-04 7.0457235e-03
  9.5292193e-01 1.3305187e-02]]
[[3.2139092e-03 1.7617320e-03 1.5156326e-02 4.5514229e-04 7.2275382e-03
  9.6333230e-01 8.8529326e-03]]
[[2.3175064e-03 9.1684898e-03 3.6709320e-03 3.5639812e-05 5.5496377e-04
  9.8203880e-01 2.2136262e-03]]
[[1.2962397e-03 2.4440070e-04 1.0581431e-03 5.7211892e-05 5.2986102e-04
  9.9559844e-01 1.2156332e-03]]
[[3.6711968e-03 2.2427528e-03 4.1232309e-03 1.9961045e-04 1.5861710e-03
  9.8315215e-01 5.0248196e-03]]
[[0.07741823 0.21544996 0.07010408 0.16967359 0.34242013 0.08667498
  0.03825898]]
[[0.11622868 0.02036616 0.03580161 0.12718418 0.2970395  0.35281828
  0.05056159]]
[[0.14951348 0.02230343 0.05702509 0.12438471 0.16366965 0.42336333
  0.05974035]]
[[0.10149433 0.0136643  0.04225045 0.0195810

[[0.19790256 0.0201329  0.02663013 0.05334774 0.34718245 0.2850709
  0.06973329]]
[[0.01302761 0.00763626 0.21121003 0.00702663 0.17121114 0.39013246
  0.19975589]]
[[0.05739162 0.00823387 0.1540708  0.00413231 0.15855058 0.20547366
  0.4121472 ]]
[[1.4202299e-03 8.2451443e-05 1.9035791e-01 1.1687580e-03 3.2718943e-03
  2.6633730e-04 8.0343235e-01]]
[[0.01062087 0.00219067 0.24370737 0.0191402  0.04825129 0.01730938
  0.6587802 ]]
[[0.04327489 0.04328373 0.1968437  0.10339703 0.16150273 0.02599449
  0.4257035 ]]
[[0.01125481 0.00589976 0.13869965 0.01834409 0.10275133 0.01284939
  0.71020097]]
[[0.00955714 0.00260739 0.09844565 0.00894374 0.10464419 0.01795629
  0.7578456 ]]
[[0.02225507 0.00376163 0.09138752 0.00717687 0.05550767 0.00460871
  0.8153025 ]]
[[0.00981926 0.00421441 0.33742857 0.00799411 0.02587659 0.04699094
  0.5676761 ]]
[[1.34940194e-02 1.07906395e-04 6.00484051e-02 1.41297886e-03
  1.07890191e-02 8.01748480e-04 9.13345873e-01]]
[[0.01358765 0.00166064 0.24044062 0.01

[[0.00776147 0.03132782 0.49478468 0.0167146  0.04164664 0.08576401
  0.32200086]]
[[0.00168969 0.01761824 0.06581887 0.16392067 0.08487918 0.01789449
  0.6481788 ]]
[[0.00330075 0.02356977 0.09737765 0.17992727 0.13112454 0.03763484
  0.5270652 ]]
[[0.00784981 0.04759187 0.22204675 0.03859301 0.22534558 0.21747155
  0.24110144]]
[[1.1661019e-03 2.6466209e-05 4.9343277e-02 6.0597964e-04 8.1020892e-03
  4.2646850e-04 9.4032961e-01]]
[[1.0023875e-03 8.3522616e-05 5.5050079e-02 1.4439812e-03 1.2823514e-02
  1.4969968e-03 9.2809951e-01]]
[[1.0030074e-03 4.4892167e-05 5.6239683e-02 1.8202104e-03 1.7182618e-02
  1.6434204e-03 9.2206621e-01]]
[[8.0539990e-04 6.0173916e-05 3.9774276e-02 2.3227935e-03 2.8918821e-02
  2.6877674e-03 9.2543077e-01]]
[[1.3427996e-03 3.2725875e-05 3.0127661e-02 3.0758088e-03 4.9824446e-02
  4.1006561e-03 9.1149592e-01]]
[[8.1777776e-04 2.0933971e-05 2.1503830e-02 1.9314451e-03 2.7448615e-02
  1.9422719e-03 9.4633514e-01]]
[[1.0803866e-03 8.4797721e-05 2.0630844e-02 

In [28]:
idx_to_class={0: 'Anger', 1: 'Disgust', 2: 'Fear', 3: 'Happiness', 4: 'Neutral', 5: 'Sadness', 6: 'Surprise'}

y_pred=np.argmax(y_scores_val,axis=1)
acc=100.0*(y_val==y_pred).sum()/len(y_val)
print('Accuracy:',acc)

y_train=np.array(training_set.labels)

for i in range(y_scores_val.shape[1]):
    _val_acc=(y_pred[y_val==i]==i).sum()/(y_val==i).sum()
    print('%s %d/%d acc: %f' %(idx_to_class[i],(y_train==i).sum(),(y_val==i).sum(),100*_val_acc))

Accuracy: 65.28599605522682
Anger 371/132 acc: 82.575758
Disgust 156/31 acc: 35.483871
Fear 251/57 acc: 43.859649
Happiness 160/19 acc: 68.421053
Neutral 153/42 acc: 28.571429
Sadness 290/59 acc: 45.762712
Surprise 643/167 acc: 80.239521


In [29]:
from sklearn.metrics import classification_report

target_names = ['Anger', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sadness', 'Surprise']
print(classification_report(y_val, y_pred, target_names=target_names))

              precision    recall  f1-score   support

       Anger       0.70      0.83      0.76       132
     Disgust       0.73      0.35      0.48        31
        Fear       0.49      0.44      0.46        57
       Happy       0.42      0.68      0.52        19
     Neutral       0.43      0.29      0.34        42
     Sadness       0.51      0.46      0.48        59
    Surprise       0.77      0.80      0.79       167

    accuracy                           0.65       507
   macro avg       0.58      0.55      0.55       507
weighted avg       0.65      0.65      0.64       507



# Making a single prediction

In [ ]:
from facial_analysis import FacialImageProcessing
imgProcessing=FacialImageProcessing(False)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential, load_model,model_from_json

my_model = load_model(r"C:\Users\HP\SLR\1src\sl_mobilenet_7_ft_sgd.h5") 
import cv2
import matplotlib.pyplot as plt
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
#mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

# For webcam input:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)
with mp_face_mesh.FaceMesh(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = face_mesh.process(image)
        
        img = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        #cv2.imshow('MediaPipe FaceMesh', img)
        
        INPUT_SIZE = (224, 224)
        idx_to_class={0: 'Anger', 1: 'Disgust', 2: 'Fear', 3: 'Happiness', 4: 'Neutral', 5: 'Sadness', 6: 'Surprise'}
        #filename = r'C:\Users\HP\SLR\test_files_1_sorted\07January_2010_Thursday_tagesschau_default-5.avi_pid0_fn000045-0.png'

        #img = plt.imread(filename)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img=cv2.resize(img,INPUT_SIZE)
        #frame_bgr=cv2.imread(filename)
        #frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(frame)
        bounding_boxes, points = imgProcessing.detect_faces(frame)
        points = points.T
        for bbox,p in zip(bounding_boxes, points):
            box = bbox.astype(np.int)
            x1,y1,x2,y2=box[0:4]    
            face_img=frame[y1:y2,x1:x2,:]

            face_img=cv2.resize(face_img,INPUT_SIZE)

            #plt.imshow(face_img)

            inp=face_img.astype(np.float32)
            inp[..., 0] -= 103.939
            inp[..., 1] -= 116.779
            inp[..., 2] -= 123.68
            inp = np.expand_dims(inp, axis=0)
            scores=my_model.predict(inp)
            #plt.title(idx_to_class[np.argmax(scores)])
            
            cv2.putText(frame, idx_to_class[np.argmax(scores)], (0, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 192, 203), 2)
            
            cv2.imshow('video', frame)
            
            #print(idx_to_class[np.argmax(scores)])

            #print(scores)

        
        if cv2.waitKey(5) & 0xFF == 27:
              break

cap.release()

In [ ]:
# For image 02August_2010_Monday_heute_default-7.avi_pid0_fn000008-0.png

INPUT_SIZE = (224, 224)
idx_to_class={0: 'Anger', 1: 'Disgust', 2: 'Fear', 3: 'Happiness', 4: 'Neutral', 5: 'Sadness', 6: 'Surprise'}
#filename = r'C:\Users\HP\SLR\test_files_1_sorted\07January_2010_Thursday_tagesschau_default-5.avi_pid0_fn000045-0.png'

#img = plt.imread(filename)
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img=cv2.resize(img,INPUT_SIZE)
#frame_bgr=cv2.imread(filename)
frame_bgr=cv2.imread(img)
frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
plt.imshow(frame)
bounding_boxes, points = imgProcessing.detect_faces(frame)
points = points.T
for bbox,p in zip(bounding_boxes, points):
    box = bbox.astype(np.int)
    x1,y1,x2,y2=box[0:4]    
    face_img=frame[y1:y2,x1:x2,:]

    face_img=cv2.resize(face_img,INPUT_SIZE)

    plt.imshow(face_img)

    inp=face_img.astype(np.float32)
    inp[..., 0] -= 103.939
    inp[..., 1] -= 116.779
    inp[..., 2] -= 123.68
    inp = np.expand_dims(inp, axis=0)
    scores=new_model.predict(inp)
    plt.title(idx_to_class[np.argmax(scores)])

    print(scores)